# Relative populations $w_{g,1}(37^\#)$
In this notebook we develop the models of relative populations of gaps $w_{g,1}(p_k^\#)$ in the cycles of gaps
$\mathcal{G}(p_k^\#)$.

We can create these models for all gaps $g < 2p_1$.  So we use counts $n_{g,j}(37^\#)$ to calculate the coefficients for the
models
$$ w_{g,1}(p_k^\#) = w_{g,1}(\infty) - l_1 \prod_{p_1}^{p_k} \frac{p-3}{p-2} + l_2 \prod_{p_1}^{p_k} \frac{p-4}{p-2} - \ldots$$
with $l_j = L_j^T \cdot n_g(37^\#)/n_{2,1}(37^\#)$.  The left eigenvectors $L^T$ are an upper triangular Pascal matrix.

For the parameter $\lambda$ for the relative population models $w_g(\lambda) = w_g(p_k^\#)$ we use the subdominant eigenvalue
$$ \lambda = \lambda(p_k) = \prod_{p_1}^{p_k} \frac{p-3}{p-2} $$

In [1]:
%reset -f

# This version uses pandas & matplotlib to provide interactive graphics
import pandas as pd
import numpy as np
from numpy.polynomial.polynomial import polyval
import array
import itertools
import matplotlib
%matplotlib inline
# matplotlib.use("nbagg")

import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
plt.ion

import gc
import psutil
import sys
import csv

We read the populatons $n_{g,j}(37^\#)$ in from a .csv file.  We have these counts for gaps $g \le 3150$ and $j \le 500$.
From the constraint $g < 2p_1$ we can only form exact models from this data for gaps $g \le 80$.
Through slightly modified calculations we can form the exact model for $g=82$ as well.

This is not quite general code.  We set bounds and sizes specific for $p_k=37$.  For example:  the maximum gap for which we can develop the exact model $w_g(p^\#)$ is $g=82$.  These models have at most $19$ terms; that is, the longest driving term for these gaps has length $J=19$.

In [2]:
dfn37 = pd.read_csv('nG37.csv')

In [3]:
sys.getsizeof(dfn37)

6449237

In [4]:
dfn37.shape

(1575, 504)

In [5]:
dfn37.iloc[0:3]

,Unnamed: 0,Totals,Ratio to Gap 2,Asymptotic Ratio to Gap 2,Length = 1,Length = 2,Length = 3,Length = 4,Length = 5,Length = 6,...,Length = 491,Length = 492,Length = 493,Length = 494,Length = 495,Length = 496,Length = 497,Length = 498,Length = 499,Length = 500
0,Gap Sum = 2,217929355875,1.0,1.0,2.179294e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gap Sum = 4,217929355875,1.0,1.0,2.179294e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gap Sum = 6,435858711750,2.0,2.0,2.939208e+11,1.419379e+11,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# we extract the population data for the gaps and their driving terms, and replace NaN with 0
ngmat=np.array(dfn37.iloc[0:41,4:23].fillna(0))

In [7]:
# pull the number of gaps g=2 for the normalization from n_g to w_g
ng2=ngmat[0,0]
ng2

np.float64(217929355875.0)

In [8]:
# block to check the available system memory
gc.collect()
memory = psutil.virtual_memory()
available_memory = memory.available
del memory
print(f"Available memory: {available_memory / (1024 ** 2):.2f} MB")

Available memory: 3012.53 MB


In [9]:
# 19x19 matrix of left eigenvectors -- an upper triangular Pascal matrix
# gaps up to g=82 have admissible driving terms of length at most J=19
eigLT=np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1,  1,  1,  1,  1,   1,   1,   1,    1,    1,    1],
                [0, 1, 2, 3, 4, 5, 6, 7, 8,  9, 10, 11, 12,  13,  14,  15,   16,   17,   18],
                [0, 0, 1, 3, 6,10,15,21,28, 36, 45, 55, 66,  78,  91, 105,  120,  136,  153],
                [0, 0, 0, 1, 4,10,20,35,56, 84,120,165,220, 286, 364, 455,  560,  680,  816],
                [0, 0, 0, 0, 1, 5,15,35,70,126,210,330,495, 715,1001,1365, 1820, 2380, 3060],
                [0, 0, 0, 0, 0, 1, 6,21,56,126,252,462,792,1287,2002,3003, 4368, 6188, 8568],
                [0, 0, 0, 0, 0, 0, 1, 7,28, 84,210,462,924,1716,3003,5005, 8008,12376,18564],
                [0, 0, 0, 0, 0, 0, 0, 1, 8, 36,120,330,792,1716,3432,6435,11440,19448,31824],
                [0, 0, 0, 0, 0, 0, 0, 0, 1,  9, 45,165,495,1287,3003,6435,12870,24310,43758],
                [0, 0, 0, 0, 0, 0, 0, 0, 0,  1, 10, 55,220, 715,2002,5005,11440,24310,48620],
                [0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  1, 11, 66, 286,1001,3003, 8008,19448,43758],
                [0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  1, 12,  78, 364,1365, 4368,12376,31824],
                [0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  1,  13,  91, 455, 1820, 6188,18564],
                [0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,   1,  14, 105,  560, 2380, 8568],
                [0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,   0,   1,  15,  120,  680, 3060],
                [0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,   0,   0,   1,   16,  136,  816],
                [0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,   0,   0,   0,    1,   17,  153],
                [0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,   0,   0,   0,    0,    1,   18],
                [0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,   0,   0,   0,    0,    0,    1]])

In [10]:
ngmat.shape

(41, 19)

In [11]:
# normalize the initial populations
w0gmat = ngmat/ng2

In [12]:
w0gmat = np.transpose(w0gmat)
w0gmat.shape

(19, 41)

In [13]:
# calculate the matrix of coefficients for the models derived from the eigenstructure
ljmat = np.matmul(eigLT,w0gmat)
# make the correction for g=2p_1=82.  We iterate forward, knowing that both boundary fusions will occur
#  in the same image, and then pull those entries back to obtain surrogate coefficients for p_0=37
#  See 2023 paper "Addendum: models for gaps g=2p_1"
ljmat[:,40]=[1.025641,11.553942,60.410483,194.488465,431.258857,697.937481,852.214506,800.382107,
              584.027075,332.122850,146.758457,49.939794,12.883935,2.461383,0.336767,0.031541,0.001910,
              0.000070,0.000001]
# associate the alternating signs with these coefficients
i=1
while (i < 19):
    ljmat[i,:] = -ljmat[i,:]
    i +=2

## Preparing to plot $w_{g,1}(\lambda)$
The matrix ljmat contains the coefficients $-l_j(g) = (-1)^{j+1} L_j^T \cdot (\frac{1}{n_{2,1}(37^\#)}n_g(37^\#))$.
In python the indices begin at $0$, so the coefficients for gap $g$ are contained in column ljmat[:, (g-2)/2].
The exact model $w_{g,1}(p^\#)$ for the relative population of $g$ in the cycle $\mathcal{G}(p^\#)$ is approximately
polynomial in $\lambda = \prod_{41}^{p_k} \frac{p-3}{p-2}$
$$ w_{g,1}(\lambda) \approx w_{g,1}(\infty) - l_2 \lambda + l_3 \lambda^2 - l_4 \lambda^3 + \cdots \; {\rm for} \; 0 < \lambda \le 1$$


In [14]:
ljmat.shape

(19, 41)

In [15]:
np.save('lj37.npy',ljmat)

In [16]:
# create an array of lambda values
lampar = np.arange(0,1,0.0005)
lampar = np.square(lampar) # These curves have more shape as lambda gets smaller, so we adjust the sampled values

In [20]:
# calculate values for the curves at the lambda values
# Note: we have to handle the model for g=82 separately, as a boundary case
wgpk = np.zeros((lampar.size, 41))
i = 0
while (i < 41):
    wgpk[:,i] = polyval(lampar, ljmat[:,i])
    i += 1


In [21]:
wgpk.shape

(2000, 41)

In [22]:
# Develop a master color dictionary - by gap
# we set colors by family, as determined by prime factors of the gap
mastercolordict={'2':'#FF0000', '4':'#FF0000', '8':'#CD5C5C', '16':'#DC143C', '32':'#C71585', '64':'#F08080',
                 '6':'#0000FF', '12':'#1E90FF', '18':'#000080', '24':'#4169E1', '36':'#00BFFF', '48':'#4682B4',
                 '54':'#7EF1F9', '72':'#6495ED',
                 '10':'#006400', '20':'#008000', '40':'#32CD32', '50':'#3CB371', '80':'#00FF00',
                 '30':'#EEBC18', '60':'#FCD247', '90':'#FFCC33',
                 '14':'#FF9A00', '28':'#FF8000', '56':'#F5C75E',
                 '42':'#FF80C8', '84':'#32CD32', '70':'#CBDF48', 
                 '22':'#0000CD', '44':'#20B2AA', '66':'#666600',
                 '26':'#0000CD', '52':'#20B2AA', '78':'#999900',
                 '34':'#0000CD', '68':'#20B2AA', '38':'#0000CD', '76':'#20B2AA',
                 '46':'#808080', '58':'#20B2AA', '62':'#808080', '74':'#20B2AA', '82':'#4682B4'}

In [23]:
smallfactorgaps = [2,4,6,8,10,12,14,16,18,20,24,28,30,32,36,40,42,48,50,54,56,60,64,70,72,80]

def draw_wg(xmax,ymin,bigfactors):
    # plotting the curves
    plt.clf()
    fig, ax = plt.subplots()
    fig.set_size_inches(12,9)
    ax.set_title(f"$w_g(p_k^\#)$ for $p_0=37$ and gaps $g < 82$")
    # set gridlines for x-axis (lambda)
    if (xmax < 0.05):
        marklampar = 0.001
    elif (xmax < 0.3):
        marklampar = 0.01
    elif (xmax < 0.75):
        marklampar = 0.05
    else:
        marklampar = 0.1
    ax.set_xticks(np.arange(0,xmax,marklampar))
    ax.grid(axis='x', color='#080408', lw=0.0625, markevery=marklampar)
    ax.grid(axis='y', color='#A9A9A9', lw=0.0625 )
    ax.set_xlim(0,xmax)
    ax.set_ylim(ymin,2.75)

    ax.plot(lampar, wgpk[:,0], color='#FF0000', lw=1.5, label='g=2, 4')

    i=2
    while (i < 41):
        gapval = 2*(i+1)
        gapstr=str(gapval)
        wgcolor = mastercolordict[gapstr]
        if (gapval in smallfactorgaps):
            ax.plot(lampar, wgpk[:,i], color=wgcolor, lw=1.5, label='g='+gapstr)
        elif (bigfactors):
            ax.plot(lampar, wgpk[:,i], color=wgcolor, lw=0.625, label='g='+gapstr)
        i += 1

    # changing the order of the legend, to group the gaps by family at asymptotic value
    handles, labels = ax.get_legend_handles_labels()
    if bigfactors:
        sortedhandles = [handles[13],handles[28],handles[19],handles[31],handles[37],
                         handles[1],handles[4],handles[7],handles[10],handles[16],handles[22],
                         handles[25],handles[34],handles[33],handles[3],handles[8],handles[18],
                         handles[23],handles[38],handles[5],handles[12],handles[26],handles[9],
                         handles[20],handles[11],handles[24],handles[15],handles[32],handles[17],
                         handles[36],handles[21],handles[27],handles[29],handles[35],
                         handles[39],handles[0],handles[2],handles[6],handles[14],handles[30]]
        sortedlabels = [labels[13],labels[28],labels[19],labels[31],labels[37],
                         labels[1],labels[4],labels[7],labels[10],labels[16],labels[22],
                         labels[25],labels[34],labels[33],labels[3],labels[8],labels[18],
                         labels[23],labels[38],labels[5],labels[12],labels[26],labels[9],
                         labels[20],labels[11],labels[24],labels[15],labels[32],labels[17],
                         labels[36],labels[21],labels[27],labels[29],labels[35],
                         labels[39],labels[0],labels[2],labels[6],labels[14],labels[30]]

        ax.legend(sortedhandles, sortedlabels, ncol=3)
    else:
        sortedhandles = [handles[11],handles[20],handles[15],handles[1],handles[4],
                         handles[7],handles[9],handles[13],handles[16],handles[18],handles[23],
                         handles[22],handles[3],handles[8],handles[14],handles[17],handles[24],
                         handles[5],handles[10],handles[19],handles[0],handles[2],handles[6],
                         handles[12],handles[21]]
        sortedlabels = [labels[11],labels[20],labels[15],labels[1],labels[4],
                         labels[7],labels[9],labels[13],labels[16],labels[18],labels[23],
                         labels[22],labels[3],labels[8],labels[14],labels[17],labels[24],
                         labels[5],labels[10],labels[19],labels[0],labels[2],labels[6],
                         labels[12],labels[21]]

        ax.legend(sortedhandles, sortedlabels, ncol=3)

# XXXQHERE: fix colors

    plt.show()

# widgets for interacting with the graph of the w_g
xmaxSelect = widgets.SelectionSlider(options=[0.01,0.05,0.1,0.15,0.2,0.3,0.5,0.75,1.0],
                                      value=1.0, description="max lambda", layout=widgets.Layout(width='60%'), disabled=False)
yminSelect = widgets.SelectionSlider(options=[0.0, 0.1, 0.25, 0.5, 0.75],
                                      value=0.0, description="min w_g", layout=widgets.Layout(width='60%'), disabled=False)
bigfactorSelect = widgets.Checkbox(value=True, description='Factors > 7') 

interact(draw_wg, xmax=xmaxSelect, ymin=yminSelect, bigfactors=bigfactorSelect)


interactive(children=(SelectionSlider(description='max lambda', index=8, layout=Layout(width='60%'), options=(…

<function __main__.draw_wg(xmax, ymin, bigfactors)>

<Figure size 640x480 with 0 Axes>

In [34]:
lampar.size

2000

In [35]:
lampar[1500]

np.float64(0.5625)